In [2]:
!pip install requests
!pip install numpy
!pip install pandas

In [1]:
import re
import requests
import numpy as np
import pandas as pd
import time

In [7]:
url = 'https://www.lamudi.co.id/jakarta/house/buy/?page=1'
r = requests.get(url)
html = r.text
for line in html.split('\n'):
    print(line)

<!DOCTYPE html>
<html>
    <head>
        <noscript>
            <title>Pardon Our Interruption</title>
        </noscript>

        <meta name="viewport" content="width=1000">
        <meta name="robots" content="noindex, nofollow">
        <meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate">
        <meta http-equiv="pragma" content="no-cache">
        <meta http-equiv="expires" content="0">

        <style>
            .container { max-width: 800px; margin: auto; font-family: 'Helvetica Neue',Helvetica,Arial,sans-serif; color: #7a838c; }
            h1 { color: #2a2d30; font-weight: 500; }
            li { margin: 0 0 10px; }
            a { color: #428bca; }
            a:hover, a:focus { color: #2a6496; }
        </style>

        <script>
            function showBlockPage() {
                document.title = "Pardon Our Interruption"
                document.getElementsByClassName("container")[0].style.display = "block";
            }
            setTi

Apparently there is a validation when trying to scrape. However, when I look to the website, there is no explicit statement about scraping prohibition. Therefore, we are going to use another route. That is, Selenium.

In [11]:
!pip install selenium
!pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [14]:
links = []
data_price_s = []
data_category_s = []
data_subcategories_s = []
data_bedrooms_s = []
data_bathrooms_s = []
data_building_size_s = []
data_land_size_s = []
data_furnished_s = []
data_sku_s = []
data_geo_point_s = []
page_link_s = []
parent_link_s = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.lamudi.co.id/jakarta/house/buy/")
click_expand = driver.find_element(By.XPATH, '//*[@id="js-crosslinkTopFilter"]/a').click()
sublinks = driver.find_elements(By.XPATH, '//*[@id="js-crosslinkTopFilter"]/div/div/div/a')
for tag in sublinks:
    links.append(tag.get_attribute('href'))
driver.quit()
for link in links: # start from 18th link since there was a problem with page in previous loop
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(link)
    page_end = 1 # initialize page_end with number 1 to be refined, only for looping purpose
    page_iter = 1
    while page_iter <= page_end:
        info = driver.find_elements(By.CSS_SELECTOR, ".ListingCell-AllInfo.ListingUnit")
        info_a = driver.find_elements(By.CSS_SELECTOR, ".js-listing-link")
        try:
            page_end = int(driver.find_element(By.CSS_SELECTOR, "*[data-pagination-end]:last-child").get_attribute('data-pagination-end'))
        except:
            pass
        for i in range(len(info)):
            data_price_s.append(info[i].get_attribute('data-price'))
            data_category_s.append(info[i].get_attribute('data-category'))
            data_subcategories_s.append(info[i].get_attribute('data-subcategories'))
            data_bedrooms_s.append(info[i].get_attribute('data-bedrooms'))
            data_bathrooms_s.append(info[i].get_attribute('data-bathrooms'))
            data_building_size_s.append(info[i].get_attribute('data-building_size'))
            data_land_size_s.append(info[i].get_attribute('data-land_size'))
            data_furnished_s.append(info[i].get_attribute('data-furnished'))
            data_sku_s.append(info[i].get_attribute('data-sku'))
            data_geo_point_s.append(info[i].get_attribute('data-geo-point'))
            page_link_s.append(info_a[i*2].get_attribute('href')) # js-listing-link class is written twice in the website
            parent_link_s.append(link)
        page_iter += 1
        if page_end >= page_iter:
            click_expand = driver.find_element(By.CSS_SELECTOR, '.next a[data-next-page]').click()
            time.sleep(2.811 + np.random.normal(loc=0.0,scale=0.627)) # delay so it doesn't break or startle the website
    driver.quit()
df = pd.DataFrame(data={'data_price_s': data_price_s,
'data_category_s': data_category_s,
'data_subcategories_s': data_subcategories_s,
'data_bedrooms_s': data_bedrooms_s,
'data_bathrooms_s': data_bathrooms_s,
'data_building_size_s': data_building_size_s,
'data_land_size_s': data_land_size_s,
'data_furnished_s': data_furnished_s,
'data_sku_s': data_sku_s,
'data_geo_point_s': data_geo_point_s,
'page_link_s': page_link_s,
'parent_link_s': parent_link_s})
df.to_csv('lamudi_house_dataset.csv')

Let's check the output

In [16]:
print(data_price_s[:100])
print(data_category_s[:100])
print(data_subcategories_s[:100])
print(data_bedrooms_s[:100])
print(data_bathrooms_s[:100])
print(data_building_size_s[:100])
print(data_land_size_s[:100])
print(data_furnished_s[:100])
print(data_sku_s[:100])
print(data_geo_point_s[:100])
print(page_link_s[:100])
print(parent_link_s[:100])

['1100000000', '980000000', '550000000', '730000000', '4500000000', '10000000000', '1800000000', '451000000', '2000000000', '451000000', '270000000', '980000000', '965000000', '2500000000', '1311000000', '950000000', '185000000', '420000000', '5300000000', '587000000', '2800000000', '17000000000', '539000000', '4500000000', '7800000000', '448000000', '6500000000', '1333000000', '1117000000', '7000000000', '8000000000', '7000000000', '980225548', '730000000', '280000000', '5300000000', '7000000000', '1035123013', '1032450038', '185000000', '17000000000', '964059202', '962456061', '963214174', '1049800566', '964132500', '962038202', '964132500', '963568000', '964523080', '971999999', '962026582', '963852030', '962069000', '1034580011', '961026205', '1033018205', '964056505', '963050505', '925808446', '6000000000', '2300000000', '1450000000', '3000000000', '2950000000', '3000000000', '1850000000', '3500000000', '3950000000', '1900000000', '1800000000', '6300000000', '1900000000', '1500000

Looks good to me!